# Feature Selection Tutorial

In this Jupyter notebook, we'll walk through the information-theoretic feature selection algorithms in PicturedRocks. 

In [1]:
import numpy as np

In [2]:
import scanpy.api as sc
import picturedrocks as pr

In [3]:
adata = sc.datasets.paul15()

... storing 'paul15_clusters' as categorical


In these next two cells, we're copying the cluster labels and processing them. This is necessary for supervised analysis 
and visualization tools in PicturedRocks that use cluster labels.

In [4]:
adata.obs['clust']  = adata.obs['paul15_clusters']
adata.obs['y'] = adata.obs['clust'].cat.codes

In [5]:
pr.read.process_clusts(adata)

AnnData object with n_obs × n_vars = 2730 × 3451 
    obs: 'paul15_clusters', 'clust', 'y'
    uns: 'iroot', 'num_clusts', 'clusterindices'

Normalize per cell and log transform the data

In [6]:
sc.pp.normalize_per_cell(adata)

In [7]:
sc.pp.log1p(adata)

The `make_infoset` method creates an `InformationSet` object with the following discretization: $[\log(X_{ij} + 1)]_{ij}$ where $X_ij$ is the data matrix. It is useful to have only a small number of discrete states that each gene can take so that entropy is a reasonable measurement.

In [8]:
infoset = pr.markers.makeinfoset(adata, True)

Altenatively, we could have made our own InformationSet object with our own discretization. Here we re-create `infoset` by hand. Note: At this time, InformationSet cannot handle scipy's sparse matrices.

In [9]:
X_disc = np.log2(adata.X + 1).round().astype(int)

In [10]:
infoset2 = pr.markers.mutualinformation.infoset.InformationSet(np.c_[X_disc, adata.obs['y']], True)

In [11]:
np.array_equal(infoset2.X, infoset.X)

True

Because this dataset only has 3451 features, it is computationally easy to do feature selection without restricting the number of features. If we wanted to, we could do either supervised or unsupervised univariate feature selection (i.e., without considering any interactions between features).

In [12]:
# supervised
mim = pr.markers.mutualinformation.iterative.MIM(infoset)
most_relevant_genes = mim.autoselect(1000)

In [13]:
# unsupervised
ue = pr.markers.mutualinformation.iterative.UniEntropy(infoset)
most_variable_genes = ue.autoselect(1000)

At this stage, if we wanted to, we could slice our `adata` object as `adata[:,most_relevant_genes]` or `adata[:,most_variable_genes]` and create a new `InformationSet` object for this sliced object. We don't need to do that here.

## Supervised Feature Selection

Let's jump straight into supervised feature selection. Here we will use the `CIFE` objective

In [14]:
cife = pr.markers.mutualinformation.iterative.CIFE(infoset)

In [15]:
cife.score

array([0.06153028, 0.03873652, 0.10514698, ..., 0.13305092, 0.17305014,
       0.03877704])

In [16]:
top_genes = np.argsort(cife.score)[::-1]
print(adata.var_names[top_genes[:10]])

Index(['Ctsg', 'Prtn3', 'Car2', 'Elane', 'Car1', 'H2afy', 'Ermap', 'Mt2',
       'Fam132a', 'Klf1'],
      dtype='object')


Let's select 'Car1'

In [17]:
ind = adata.var_names.get_loc('Car1')

In [18]:
cife.add(ind)

Now, the top genes are

In [19]:
top_genes = np.argsort(cife.score)[::-1]
print(adata.var_names[top_genes[:10]])

Index(['Elane', 'Ctsg', 'Prtn3', 'Apoe', 'Fam132a', 'P4hb', 'Gstm1', 'H2afy',
       'Alas1', 'Car2'],
      dtype='object')


Observe that the order has changed based on redundancy (or lack thereof) with 'Car1'. Let's add 'Ctsg'

In [20]:
ind = adata.var_names.get_loc('Ctsg')
cife.add(ind)

In [21]:
top_genes = np.argsort(cife.score)[::-1]
print(adata.var_names[top_genes[:10]])

Index(['Apoe', 'Mt1', 'Mcm5', 'Ran', 'Srm', 'Uqcrq', 'Psma7', 'Hspd1', 'Uqcr',
       'Cox5a'],
      dtype='object')


If we want to select the top gene at each step, we can use `autoselect`

In [22]:
cife.autoselect(10)

To look at the markers we've selected, we can examine `cife.S`

In [23]:
cife.S

[552, 815, 305, 1953, 3002, 2666, 2479, 596, 2645, 785, 412, 1503]

In [24]:
adata.var_names[cife.S]

Index(['Car1', 'Ctsg', 'Apoe', 'Mt1', 'Taldo1', 'S100a10', 'Ptprcap', 'Ccnd2',
       'Rps8', 'Crip1', 'Atpase6', 'Hspa8'],
      dtype='object')

This process can also done manually with a user-interface.

In [25]:
im = pr.markers.interactive.InteractiveMarkerSelection(adata, cife)

In [26]:
im.show()

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'VBox(children=(HBox(children=(VBox(), …

Note, that because we passed the same `cife` object, any genes added/removed in the interface will affect the `cife` object.

In [27]:
adata.var_names[cife.S]

Index(['Car1', 'Ctsg', 'Apoe', 'Mt1', 'Taldo1', 'S100a10', 'Ptprcap', 'Ccnd2',
       'Rps8', 'Crip1', 'Atpase6', 'Hspa8'],
      dtype='object')

## Unsupervised Feature Selection

This works very similarly. In the example below, we'll autoselect 5 genes and then run the interface. Note that although the previous section would not work without cluster labels, the following code will.

In [28]:
cife_unsup = pr.markers.mutualinformation.iterative.CIFEUnsup(infoset)

In [29]:
cife_unsup.autoselect(5)

(If you ran the example above, this will load faster because the t_SNE coordinates for genes and cells have already been computed. You can also disable one/both of these plots with keyword arguments (e.g., `InteractiveMarkerSelection(..., show_genes=False)`)

In [30]:
im_unsup = pr.markers.interactive.InteractiveMarkerSelection(adata, cife_unsup, show_genes=False)

In [31]:
im.show()

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'VBox(children=(HBox(children=(VBox(), …